# CCL precision of cluster lensing quantities

The "out-of-the-box" numerical integration of CCL to compute the Sigma and DeltaSigma profiles relevant to cluster lensing do not reach the same level of precision as NumCosmo or Colossus. In this NB, we use the NFW profile to explore the precision parameters available in CCL; the goal is to see if we can get a better match between the quantities integrated numerically and the analytical solution available.

This uses CCL only. Comparison with NumCosmo and Colossus using the default parameters is available in the `validation_tests.ipynb` notebook in this directory.

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import pyccl as ccl

In [ ]:
# CCL Cosmology
cosmo_ccl = ccl.Cosmology(Omega_c=0.2621, Omega_b=0.049, h=0.6766, n_s=0.9665, sigma8=0.8102)

In [ ]:
print(cosmo_ccl)

In [ ]:
Delta = 200
Mvir  = 1.0e15 # M_sun
cvir  = 5.0
z     = 0.2
a     = 1.0 / (1.0 + z)

# CCL
MDEF               = 'matter'
mdef               = ccl.halos.MassDef (Delta, MDEF)
conc               = ccl.halos.ConcentrationConstant(cvir)
mdef.concentration = conc

ccl_nfw_num            = ccl.halos.HaloProfileNFW (conc,
                                               truncated = False, 
                                               projected_analytic = False, 
                                               cumul2d_analytic = False)
ccl_nfw_ana            = ccl.halos.HaloProfileNFW (conc,
                                               truncated = False, 
                                               projected_analytic = True, 
                                               cumul2d_analytic = True)

In [ ]:
r = 10**np.arange (0, 4, 0.02) * 1.0e-3 # Mpc - physical

### 0. List of precision parameters available

In [ ]:
ccl_nfw_num.precision_fftlog

### 1.`n_per_decade` parameter --> can improve things marginally

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1,figsize=(10,8))
ax.set_xscale ('log')
ax.set_yscale ('log')  
ax.set_xlabel('R [Mpc]', fontsize=16)
ax.set_ylabel(r'$|\Sigma_{\mathrm{i}} / \Sigma_{\mathrm{ccl, ana.}} - 1|$', fontsize=16)

for n in np.arange(1000,11000,3000):
    ccl_nfw_num.update_precision_fftlog (n_per_decade = n)

    ccl_Sigma_nfw_ana = ccl_nfw_ana.projected (cosmo_ccl, r / a, Mvir, a, mdef) / a**2 
    ccl_Sigma_nfw_num = ccl_nfw_num.projected (cosmo_ccl, r / a, Mvir, a, mdef) / a**2

    ax.plot (r, np.abs (ccl_Sigma_nfw_num / ccl_Sigma_nfw_ana - 1.0), 
             label = "CCL (num), n_per_decade = "+str(n), lw=1)

ax.legend()
    

### 2.`padding_*_fftlog` parameters --> remove oscillations

In [ ]:
#reset n_per_decade to default
ccl_nfw_num.update_precision_fftlog (n_per_decade = 1000) # default

fig, ax = plt.subplots(ncols=1, nrows=1,figsize=(10,8))
ax.set_xscale ('log')
ax.set_yscale ('log')  
ax.set_xlabel('R [Mpc]', fontsize=16)
ax.set_ylabel(r'$|\Sigma_{\mathrm{i}} / \Sigma_{\mathrm{ccl, ana.}} - 1|$', fontsize=16)

for fact in np.logspace(2,4,3):
    ccl_nfw_num.update_precision_fftlog (padding_lo_fftlog=1./fact)
    ccl_nfw_num.update_precision_fftlog (padding_hi_fftlog=fact)


# CCL
    ccl_Sigma_nfw_ana = ccl_nfw_ana.projected (cosmo_ccl, r / a, Mvir, a, mdef) / a**2
    ccl_Sigma_nfw_num = ccl_nfw_num.projected (cosmo_ccl, r / a, Mvir, a, mdef) / a**2

    ax.plot (r, np.abs (ccl_Sigma_nfw_num / ccl_Sigma_nfw_ana - 1.0), 
             label = "CCL (num), padding_fftlog = "+str(fact), lw=1)

ax.legend()

### 3. `padding_*_extra` paramters --> no effect

In [ ]:
#reset padding to default
ccl_nfw_num.update_precision_fftlog (padding_lo_fftlog = 0.01)
ccl_nfw_num.update_precision_fftlog (padding_hi_fftlog = 100)

fig, ax = plt.subplots(ncols=1, nrows=1,figsize=(10,8))
ax.set_xscale ('log')
ax.set_yscale ('log')    
ax.set_xlabel('R [Mpc]', fontsize=16)
ax.set_ylabel(r'$|\Sigma_{\mathrm{i}} / \Sigma_{\mathrm{ccl, ana.}} - 1|$', fontsize=16)

for fact in np.logspace(1,3,5):
    ccl_nfw_num.update_precision_fftlog (padding_lo_extra=1./fact)
    ccl_nfw_num.update_precision_fftlog (padding_hi_extra=fact)


# CCL
    ccl_Sigma_nfw_ana = ccl_nfw_ana.projected (cosmo_ccl, r / a, Mvir, a, mdef) / a**2
    ccl_Sigma_nfw_num = ccl_nfw_num.projected (cosmo_ccl, r / a, Mvir, a, mdef) / a**2

    ax.plot (r, np.abs (ccl_Sigma_nfw_num / ccl_Sigma_nfw_ana - 1.0), 
             label = "CCL (num)", linestyle='--', lw=1)


### 4.`extrapol` parameter --> no effect

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1,figsize=(10,8))
ax.set_xscale ('log')
ax.set_yscale ('log')  
ax.set_xlabel('R [Mpc]', fontsize=16)
ax.set_ylabel(r'$|\Sigma_{\mathrm{i}} / \Sigma_{\mathrm{ccl, ana.}} - 1|$', fontsize=16)

for ep in ['linx_logy','logx_logy','linx_liny']:
    ccl_nfw_num.update_precision_fftlog (extrapol=ep)
    print(ep)

# CCL
    ccl_Sigma_nfw_ana = ccl_nfw_ana.projected (cosmo_ccl, r / a, Mvir, a, mdef) / a**2
    ccl_Sigma_nfw_num = ccl_nfw_num.projected (cosmo_ccl, r / a, Mvir, a, mdef) / a**2

    ax.plot (r, np.abs (ccl_Sigma_nfw_num / ccl_Sigma_nfw_ana - 1.0), 
             label = "CCL (num)", linestyle='--', lw=1)


### 5.`plaw_projected` parameter --> some effect
The default value seems to be the best compromise for acceptable behaviour at low and high radii.

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1,figsize=(10,8))
ax.set_xscale ('log')
ax.set_yscale ('log')  
ax.set_xlabel('R [Mpc]', fontsize=16)
ax.set_ylabel(r'$|\Sigma_{\mathrm{i}} / \Sigma_{\mathrm{ccl, ana.}} - 1|$', fontsize=16)

for p in [-1.5,-0.5, -1]:
    ccl_nfw_num.update_precision_fftlog (plaw_projected=p)

# CCL
    ccl_Sigma_nfw_ana = ccl_nfw_ana.projected (cosmo_ccl, r / a, Mvir, a, mdef) / a**2
    ccl_Sigma_nfw_num = ccl_nfw_num.projected (cosmo_ccl, r / a, Mvir, a, mdef) / a**2

    ax.plot (r, np.abs (ccl_Sigma_nfw_num / ccl_Sigma_nfw_ana - 1.0), 
             label = "CCL (num), plaw_projected = "+str(p), lw=1)
ax.legend()


### 6.`plaw_fourier` parameter --> no effect

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1,figsize=(10,8))
ax.set_xscale ('log')
ax.set_yscale ('log')  
ax.set_xlabel('R [Mpc]', fontsize=16)
ax.set_ylabel(r'$|\Sigma_{\mathrm{i}} / \Sigma_{\mathrm{ccl, ana.}} - 1|$', fontsize=16)

for p in [-2.5,-1.5,-2]:
    ccl_nfw_num.update_precision_fftlog (plaw_fourier=p)

    ccl_Sigma_nfw_ana = ccl_nfw_ana.projected (cosmo_ccl, r / a, Mvir, a, mdef) / a**2
    ccl_Sigma_nfw_num = ccl_nfw_num.projected (cosmo_ccl, r / a, Mvir, a, mdef) / a**2

    ax.plot (r, np.abs (ccl_Sigma_nfw_num / ccl_Sigma_nfw_ana - 1.0), 
             label = "CCL (num), plaw_fourier = "+str(p), linestyle='--', lw=1)
ax.legend()
